In [1]:
import pandas as pd
from sklearn import ensemble
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing

columnName = ['seed', 'lat', 'long'	, 'fg2pct', 'fg3pct', 'ftpct', 'blockpct', 'oppfg2pct', 'oppfg3pct', 'oppftpct',
              'oppblockpct', 'f3grate', 'oppf3grate', 'arate', 'opparate', 'stlrate', 'oppstlrate', 'tempo', 'adjtempo',
              'oe', 'adjoe', 'de', 'adjde', 'rpi_rating', 'pt_school_ncaa', 'pt_overall_ncaa', 'pt_school_s16', 'pt_overall_s16',
              'pt_school_ff', 'pt_overall_ff', 'pt_career_school_wins',	'pt_career_school_losses', 'pt_career_overall_losses',
              'team_season_wins', 'pt_team_season_losses', 'pt_coach_season_wins', 'pt_coach_season_losses']

df = pd.read_csv('NCAA_Tourney_2002-2016_1.csv')
features = df.iloc[:, :-4]
results = df.iloc[:, -4:]

df_team1 = df.iloc[:, 2:39]
df_team1.columns = columnName
df_team2 = df.iloc[:, 39:76]
df_team2.columns = columnName

df_team1.loc[:, 'expwin'] = df.loc[:, 'team1_adjoe'] ** 11.5 / (df.loc[:, 'team1_adjde'] ** 11.5
                                                            + df.loc[:, 'team1_adjoe'] ** 11.5)


df_team2.loc[:, 'expwin'] = df.loc[:, 'team2_adjoe'] ** 11.5 / (df.loc[:, 'team2_adjde'] ** 11.5
                                                            + df.loc[:, 'team2_adjoe'] ** 11.5)


df_team1.loc[:, 'log5'] = (df_team1.loc[:, 'expwin'] - df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']) / \
            ((df_team1.loc[:, 'expwin'] + df_team2.loc[:, 'expwin'] - 2* df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']))

df_team2.loc[:, 'log5'] = (df_team2.loc[:, 'expwin'] - df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']) / \
            ((df_team1.loc[:, 'expwin'] + df_team2.loc[:, 'expwin'] - 2* df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']))


data = df_team1 - df_team2

#normolizer = preprocessing.Normalizer().fit(data)
#df_norm = pd.DataFrame(normolizer.transform(data))
#df_norm.columns = ['seed', 'lat', 'long'	'fg2pct', 'fg3pct', 'ftpct', 'blockpct', 'oppfg2pct', 'oppfg3pct', 'oppftpct',
#              'oppblockpct', 'f3grate', 'oppf3grate', 'arate', 'opparate', 'stlrate', 'oppstlrate', 'tempo', 'adjtempo',
#              'oe', 'adjoe', 'de', 'adjde', 'rpi_rating', 'pt_school_ncaa', 'pt_overall_ncaa', 'pt_school_s16', 'pt_overall_s16',
#              'pt_school_ff', 'pt_overall_ff', 'pt_career_school_wins',	'pt_career_school_losses', 'pt_career_overall_losses',
#              'team_season_wins', 'pt_team_season_losses', 'pt_coach_season_wins', 'pt_coach_season_losses', 'expwin','log5']

X = data
y = results.loc[:, 'result']
X_train = pd.DataFrame()
X_test = pd.DataFrame()
y_train = pd.DataFrame()
y_test = pd.DataFrame()
X_valid = pd.DataFrame()
y_valid = pd.DataFrame()


for year in range(2002, 2016):
    X_train = pd.concat([X_train, X.loc[results.Season == year]])
    y_train = pd.concat([y_train, y.loc[results.Season == year]])

for year in range(2016, 2017):
    X_test = pd.concat([X_test, X.loc[results.Season == year]])
    y_test = pd.concat([y_test, y.loc[results.Season == year]])
    
paramsGB = {'n_estimators': 10, 'max_depth': 5, 'min_samples_split': 2, 'loss': 'deviance'}

original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': 5, 'random_state': 2,
                   'min_samples_split': 5}

clf = ensemble.GradientBoostingClassifier(**original_params)

clf.fit(X_train, y_train.values.ravel())

y_pred = clf.predict(X_test)

sns.set(font_scale=1, rc={"lines.linewidth": 1.2})

fig = plt.figure(figsize=(20, 10), dpi=80)
fig.suptitle('Predictor Importance', fontsize=20)

plt.subplot(1,1,1)

f = clf.feature_importances_
%matplotlib
f = 100.0 * (f / f.max())
indices = np.argsort(f)[-30:]
plt.barh(np.arange(30), f[indices],color='dodgerblue',alpha=.4)
plt.yticks(np.arange(30 + 0.25), np.array(X.columns)[indices])
plt.xlabel('Relative importance'), \
plt.title('Final Selection for Important Predictors', fontsize=20)

Using matplotlib backend: MacOSX


(<matplotlib.text.Text at 0x11ab6deb8>, <matplotlib.text.Text at 0x11ae8c320>)

In [8]:
from sklearn import linear_model
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

testResult = list()

for i in range(20,35):
    columns = list(X.columns[np.argsort(f)[-i:]])
    newFeatures = data.loc[:, columns]
    X_new = newFeatures
    y_new = results.loc[:, 'result']
    
    
    for testSize in range(1, 10):
    
        X_train, X_test, y_train, y_test = train_test_split(newFeatures, y_new, test_size=testSize/10)
        
        X_train, X_valid, y_train, y_valid = train_test_split(X_train.copy(), y_train.copy(), test_size=0.3)
        
        clf = linear_model.LogisticRegressionCV(
            Cs=list(np.power(10.0, np.arange(-10,10))), 
            penalty='l2',
            cv=10,
            solver='liblinear', max_iter=500, tol=1e-6, scoring='neg_log_loss')    
        
        
        for p in range(3,i):
            pca = PCA(p)
        
            pipeline = make_pipeline(pca, clf)
            pipeline.fit(X_train, y_train)
            
            sig_clf = CalibratedClassifierCV(pipeline, method="sigmoid", cv='prefit')
            sig_clf.fit(X_valid, y_valid.values.ravel())
            
            y_pred = sig_clf.predict_proba(X_test)
            logLoss = log_loss(y_true=y_test, y_pred=y_pred)
            
            print(logLoss, '\t\t', i, '\t\t', testSize, '\t\t', p)
            
            testResult.append([logLoss, i, testSize, p])

0.649756514818 		 20 		 1 		 3


0.649114958256 		 20 		 1 		 4


0.663012754713 		 20 		 1 		 5


0.664463920242 		 20 		 1 		 6


0.601158682895 		 20 		 1 		 7


...

0.599859525667 		 34 		 9 		 31


/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


0.599859536018 		 34 		 9 		 32


/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


0.599859539229 		 34 		 9 		 33


In [9]:
from sklearn import linear_model
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

testResult = list()

for i in range(10,35):
    columns = list(X.columns[np.argsort(f)[-i:]])
    newFeatures = data.loc[:, columns]
    X_new = newFeatures
    y_new = results.loc[:, 'result']
    
    X_train = pd.DataFrame()
    X_test = pd.DataFrame()
    y_train = pd.DataFrame()
    y_test = pd.DataFrame()
    
    for year in range(2002, 2016):
        X_train = pd.concat([X_train, X.loc[results.Season == year]])
        y_train = pd.concat([y_train, y.loc[results.Season == year]])
    
    for year in range(2016, 2017):
        X_test = pd.concat([X_test, X.loc[results.Season == year]])
        y_test = pd.concat([y_test, y.loc[results.Season == year]])
    
    
    
    #X_train, X_test, y_train, y_test = train_test_split(newFeatures, y_new, test_size=0.1)
        
    X_valid = X_train.copy()
    y_valid = y_train.copy()
    
    clf = linear_model.LogisticRegressionCV(
            Cs=2,#list(np.power(10.0, np.arange(-10,10))), 
            penalty='l2',
            cv=10,
            solver='liblinear', max_iter=500, tol=1e-6, scoring='neg_log_loss')    
        
    for p in range(3,i):
        pca = PCA(p)
        
        pipeline = make_pipeline(pca, clf)
        pipeline.fit(X_train, y_train)
        
        sig_clf = CalibratedClassifierCV(pipeline, method="sigmoid", cv='prefit')            
        sig_clf.fit(X_valid, y_valid.values.ravel())
            
        y_pred = sig_clf.predict_proba(X_test)
        logLoss = log_loss(y_true=y_test, y_pred=y_pred)
            
        print(logLoss, '\t\t', i, '\t\t', p)
            
        testResult.append([logLoss, i, p])

/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.798536989729 		 10 		 3
0.792606698355 		 10 		 4


/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:415: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:425: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:427: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:415: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])


0.570522285158 		 10 		 5


/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:415: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:425: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:427: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:415: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn

...

0.599257609916 		 34 		 31


/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:415: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:425: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:427: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:415: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.597057540489 		 34 		 32


0.59704703603 		 34 		 33


/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:415: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:425: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:427: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
/Users/maxwelllee54/anaconda/lib/python3.5/site-packages/sklearn/calibration.py:415: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])


In [4]:
m = pd.DataFrame(testResult, columns=['log_loss', 'no_predictor', 'no_pca'])
m.to_csv('log_loss_matrix3.csv')

In [10]:
m = pd.read_csv('log_loss_matrix.csv').iloc[:,1:]

In [12]:
import matplotlib.projections as proj
from matplotlib.colors import colorConverter
from mpl_toolkits.mplot3d.axis3d import Axis
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


class axis3d_custom(Axis):
    def __init__(self, adir, v_intervalx, d_intervalx, axes, *args, **kwargs):
        Axis.__init__(self, adir, v_intervalx, d_intervalx, axes, *args, **kwargs)
        self.gridline_colors = []
    def set_gridline_color(self, *gridline_info):
        '''Gridline_info is a tuple containing the value of the gridline to change
        and the color to change it to. A list of tuples may be used with the * operator.'''
        self.gridline_colors.extend(gridline_info)
    def draw(self, renderer):
        # filter locations here so that no extra grid lines are drawn
        Axis.draw(self, renderer)
        which_gridlines = []
        if self.gridline_colors:
            locmin, locmax = self.get_view_interval()
            if locmin > locmax:
                locmin, locmax = locmax, locmin

            # Rudimentary clipping
            majorLocs = [loc for loc in self.major.locator() if
                         locmin <= loc <= locmax]
            for i, val in enumerate(majorLocs):
                for colored_val, color in self.gridline_colors:
                    if val == colored_val:
                        which_gridlines.append((i, color))
            colors = self.gridlines.get_colors()
            for val, color in which_gridlines:
                colors[val] = colorConverter.to_rgba(color)
            self.gridlines.set_color(colors)
            self.gridlines.draw(renderer, project=True)

class XAxis(axis3d_custom):
    def get_data_interval(self):
        'return the Interval instance for this axis data limits'
        return self.axes.xy_dataLim.intervalx

class YAxis(axis3d_custom):
    def get_data_interval(self):
        'return the Interval instance for this axis data limits'
        return self.axes.xy_dataLim.intervaly

class ZAxis(axis3d_custom):
    def get_data_interval(self):
        'return the Interval instance for this axis data limits'
        return self.axes.zz_dataLim.intervalx

class Axes3D_custom(Axes3D):
    """
    3D axes object.
    """
    name = '3d_custom'

    def _init_axis(self):
        '''Init 3D axes; overrides creation of regular X/Y axes'''
        self.w_xaxis = XAxis('x', self.xy_viewLim.intervalx,
                            self.xy_dataLim.intervalx, self)
        self.xaxis = self.w_xaxis
        self.w_yaxis = YAxis('y', self.xy_viewLim.intervaly,
                            self.xy_dataLim.intervaly, self)
        self.yaxis = self.w_yaxis
        self.w_zaxis = ZAxis('z', self.zz_viewLim.intervalx,
                            self.zz_dataLim.intervalx, self)
        self.zaxis = self.w_zaxis

        for ax in self.xaxis, self.yaxis, self.zaxis:
            ax.init3d()
proj.projection_registry.register(Axes3D_custom)

In [15]:
import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import scipy.ndimage
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):

    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

x1 = np.linspace(m['no_predictor'].min(), m['no_predictor'].max(), len(m['no_predictor'].unique()))
y1 = np.linspace(m['no_pca'].min(), m['no_pca'].max(), len(m['no_pca'].unique()))

x2, y2 = np.meshgrid(x1, y1)

z2 = griddata((m['no_predictor'], m['no_pca']), m['log_loss'], (x2, y2), method='cubic')

mask = np.isnan(z2)
z2[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), z2[~mask])


fig = plt.figure(figsize=(24,10))
#ax = fig.gca(projection='3d')

ax = fig.add_subplot(111, projection='3d_custom')

surf = ax.plot_surface(x2, y2, z2, rstride=1, cstride=1, cmap='RdYlBu',
                       linewidth=0, antialiased=True)
ax.set_zlim(0.30, 0.80)

ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

a = Arrow3D([29, 30], [7, 10], [0.43, 0.465], mutation_scale=20,
            lw=1, arrowstyle="simple", color="r")
ax.add_artist(a)
ax.text(28, 5, 0.42, "Best LogLoss Result: 0.46", color='red')
ax.xaxis.set_gridline_color((30, 'blue'))
ax.text(25, -5, 0.39, "Predictor Number: 30", color='blue')
ax.yaxis.set_gridline_color((10, 'green'))
ax.text(5, 9, 0.37, "PCA Number: 10", color='green')

fig.colorbar(surf, shrink=0.5, aspect=5)
plt.title('Predictor and PCA Grid Search on Best Log Loss Result', fontsize=20)

plt.show()